In [37]:
import json
import pandas as pd
import numpy as np
import requests
import time
import datetime as dt

In [65]:
#do I want to update from a preexisting csv?
update = False

In [66]:
def epoch_to_date(epoch_time):
    return dt.datetime.fromtimestamp(epoch_time).strftime('%Y-%m-%d %H:%M:%S')

def date_to_epoch(df_date):
    pattern = '%Y-%m-%d %H:%M:%S'
    return int(time.mktime(time.strptime(df_date, pattern)))

In [67]:
if update==True:
    #imports previous csv
    df = pd.read_csv('./Data/Reddit_PushShift.csv',index_col=0)    
    
    #finds 24h before most recent value
    ls_date = date_to_epoch(df.sort_values('date',ascending=False).iloc[0].date)-86400 
    #reassigns df to df before 24h before last update
    #as to assume there may have been changes to 24h posts
    df = df[df['date'] > epoch_to_date(ls_date)]

    #uses request to fine date before NOW and after (24h before)Last value
    def rq(use):
        return requests.get("https://api.pushshift.io/reddit/search/submission/"+
             "?subreddit=Bitcoin&size=500&"+
             "is_video=False&"
             "before={}&after={}".format(use,ls_date))
    
else:
    df = pd.DataFrame()
    def rq(use):
        return requests.get("https://api.pushshift.io/reddit/search/submission/"+
             "?subreddit=Bitcoin&size=500&"+
             "is_video=False&"
             "before={}".format(use))

In [68]:
use = round(time.time())
init = True
n_ls = []

while (init or len(r.json()['data'])>0):
    init = False
    print('Searching Dates before {}'.format(epoch_to_date(use)))
    r = rq(use)
    #pushshifts limit is 200r/min
    # @ 1.5 sec, we are under 100r/min
    time.sleep(1.5)

    if (len(r.json()['data'])==0 or ls_date>use): break

    for sub in r.json()['data']:
        n_dic = {}
        n_dic['date'] = epoch_to_date(sub['created_utc'])
        n_dic['author'] = sub['author']
        n_dic['comments'] = sub['num_comments']
        n_dic['score'] = sub['score']
        n_dic['title'] = sub['title']
        n_dic['url'] = sub['url']
        n_ls.append(n_dic)
    df = df.append(pd.DataFrame(n_df))

    use = date_to_epoch(n_ls[-1]['date'])

Searching Dates before 2018-08-19 17:05:37
Searching Dates before 2018-08-17 15:42:33


In [69]:
df.to_csv('./Data/Reddit_PushShift.csv')